In [5]:
import findspark
findspark.init()

In [6]:
import pyspark
from pyspark.sql import SparkSession, DataFrame
from pyspark import SparkContext

In [7]:
c = SparkContext.getOrCreate()
spark = SparkSession.builder \
    .appName("Cleaning data from review csv") \
    .master("local[*]") \
    .getOrCreate()
    


In [8]:
review_df = spark.read.csv("/app/airflow/ELT-prj/files/Reviews.csv", header=True, inferSchema=True)

In [9]:
products = spark.read.csv("/app/airflow/ELT-prj/files/product_cleaned.csv", header=True, inferSchema=True)
clients = spark.read.csv("/app/airflow/ELT-prj/files/clients.csv", header=True, inferSchema=True)

In [10]:
products.printSchema()
clients.printSchema()


root
 |-- product_id: string (nullable = true)
 |-- level_1: string (nullable = true)
 |-- level_2: string (nullable = true)
 |-- level_3: string (nullable = true)
 |-- level_4: string (nullable = true)
 |-- brand_name: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_price: double (nullable = true)

root
 |-- client_id: string (nullable = true)
 |-- first_issue_date: timestamp (nullable = true)
 |-- first_redeem_date: timestamp (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)



In [11]:
# lấy các giá trị unique và nhét vào một bảng
# customer_id = products['product_id'].unique()   Cách này sai
products_id = products.select('product_id').distinct().collect() #collect để lấy về
print(type(products_id))
print(products_id[0])

<class 'list'>
Row(product_id='01e0bab3d9')


In [12]:
products_id_list = [row['product_id'] for row in products_id]
print(products_id_list[0])

01e0bab3d9


In [13]:
clients_id = clients.select('client_id').distinct().collect() #collect để lấy về
print(type(clients_id))
print(clients_id[0])

<class 'list'>
Row(client_id='001ecff0a8')


In [14]:
clients_id_list = [row['client_id'] for row in clients_id]
print(clients_id_list[0])

001ecff0a8


In [15]:
review_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- UserId: string (nullable = true)
 |-- ProfileName: string (nullable = true)
 |-- HelpfulnessNumerator: string (nullable = true)
 |-- HelpfulnessDenominator: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Text: string (nullable = true)



In [16]:
import random
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [17]:
# Cách làm này lỗi
random_product_udf = udf(lambda: random.choice(products_id_list), StringType())
random_user_udf = udf(lambda: random.choice(clients_id_list), StringType())

In [18]:
# cũng lỗi
review_df_randomized = review_df \
    .withColumn("ProductId", random_product_udf()) \
    .withColumn("UserId", random_user_udf())

# Xem thử kết quả
review_df_randomized.show(5)

+---+----------+----------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+
| Id| ProductId|    UserId|         ProfileName|HelpfulnessNumerator|HelpfulnessDenominator|Score|      Time|             Summary|                Text|
+---+----------+----------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+
|  1|48cdb36766|bacfeec422|          delmartian|                   1|                     1|    5|1303862400|Good Quality Dog ...|I have bought sev...|
|  2|2774edc10e|bfe4108226|              dll pa|                   0|                     0|    1|1346976000|   Not as Advertised|"Product arrived ...|
|  3|d52aa82ba5|191ef66ab1|"Natalia Corres "...|                   1|                     1|    4|1219017600|"""Delight"" says...|"This is a confec...|
|  4|449e6e957b|ccb663a113|                Karl|                   3|                   

In [19]:
from pyspark.sql.functions import array, lit, rand, floor, size, element_at

In [20]:
from pyspark.sql import Row
from pyspark.sql.functions import rand

# Chuyển list thành DataFrame
products_df = spark.createDataFrame([Row(product_id=x) for x in products_id_list])
clients_df = spark.createDataFrame([Row(user_id=x) for x in clients_id_list])

In [21]:
# not runnable
random_product = products_id_df.orderBy(rand()).limit(1)
random_client = clients_df.orderBy(rand()).limit(1)

NameError: name 'products_id_df' is not defined

In [22]:
spark.conf.set("spark.sql.crossJoin.enabled", True)
spark.conf.set("spark.hadoop.fs.file.impl.disable.cache", True)


In [23]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import random

random_product_udf = udf(lambda: random.choice(products_id_list), StringType())
random_client_udf = udf(lambda: random.choice(clients_id_list), StringType())

review_df_randomized = review_df \
    .withColumn("ProductId", random_product_udf()) \
    .withColumn("UserId", random_client_udf())


In [26]:

review_df_randomized.coalesce(1).write \
    .option("header", True) \
    .mode("overwrite") \
    .csv("/app/airflow/ELT-prj/files/review_randomized")


Py4JJavaError: An error occurred while calling o98.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:6